# Utilizing advanced features - Joining <ins>tests</ins> for HPC submissions
In this tutorial we will be exploring how to use advanced features of the framework. We will be using common terminology found in the repo's README.md - refer to that for any underlined terms that need clarification. Additionally, we will be building upon the material covered in the [Advanced Test Config - HPC argpacks](./AdvancedTestConfig_hpc_argpacks.ipynb); please review that tutorial if you haven't already. Anything in `"code-quoted"` format refers specifically to the test config, and anything in <ins>underlined</ins> format refers to specific terminology that can be found in the [README.md](../README.md).


In [ ]:
# Get notebook location
shellReturn = !pwd
notebookDirectory = shellReturn[0]
print( "Working from " + notebookDirectory )

Advanced usage of the <ins>run script</ins> command line option `-j` will be the focus of this tutorial :

In [ ]:
%%bash -s "$notebookDirectory"
$1/../.ci/runner.py $1/../our-config.json -h | \
  tr $'\n' '@' | \
  sed -e 's/[ ]\+-h.*\?entire suite from/.../g' | \
  sed -e 's/[ ]\+-alt.*/.../g' | \
  tr '@' $'\n'


### Joining Tests for Single HPC Submission

Tests can specify general resource usage and steps can further refine those requirements. This works well to individually submit each step independently to an HPC grid. However, for relatively small steps or if the smallest resource allocations are comparably large (e.g. whole node allocations for large core-count CPUs) one may want to aggregate the tests into larger workloads to be more effective with the resources. 

This can also be an appealing option if queue times on the grids are long, requiring individually submitted steps to each wait in the queue. Thus for steps with dependencies this would emulate re-entering the queue multiple times.

To avoid the need to over-spec the test suite to one particular machine (remember we want this to remain generic and flexible) the framework natively supports joining tests and steps into single job submissions. Tests and steps should now be able to remain as small logically separate components, steering away from machine-specific large and fragile multiple tests in one script designs.

<div class="alert alert-block alert-info">
<b>Check the FQDN of the node if using host-specific selections!</b>
When using cummulative join features for HPC runs the host that actually runs the final individual step is actually the HPC node and not the login node from which the command was launched. Joining tests under one HPC job bundles all the specified tests and their respective steps into one HPC launch step that then runs the tests normally in the node. This can at times lead to different FQDN naming between where the initial launch of the job and where the step is finally run, depending on how your computing system had been configured.
<br><br>
    
Final <ins>step argacks</ins> to the step only rely on the location where the step script is started (i.e. when "Running command" for that step is shown). This cah be affected by using the joining capabilities as the selected <ins>submit options</ins> may differ between where you launch and where it runs.

The <ins>hpc argpacks</ins> required for the steps to be submitted to the grid, total aggregated via joining or submitted normally, will only ever rely on the host that starts the test suite (the HPC login node). They are never used in the node environment for the fully joined submission and thus are not affected by changes in FQDN.
</div>

#### Accumulate Resources
Recall that each step in the end specifies its own individual set of `"arguments"` from its cummulative ancestry. The `"resources"` string field works the same way as all keys under `"submit_options"` do. The difference here is that `"resoursces"` is slightly more complex dictionary as opposed to a single dictionary of lists.

As HPC systems can be very different in scheduler args, format, and resources available to request the simplest approach is to only carry the single string. Internally, when steps and tests are joined based on scheduler type the framework makes a best guess as to how resoursces should be combined and aggregated.

The join option allows overriding specific 


